# This notebook is for the drug/prescription information.

In [ ]:
import pandas as pd
import numpy as np
import time

In [ ]:
drug_file = pd.read_csv('DRUG20Q3.txt', delimiter='$')

In [ ]:
### quantify total missing values 


null_df = pd.DataFrame(drug_file.isnull().sum().sort_values(ascending=False)).reset_index()
null_df.columns = [['column', 'missing_values']]

null_df

In [ ]:
### identify data types 

drug_file.info()

# Handling Missing Values

In [ ]:
### Convert all values to string, then match indicies for all nan and non-nan values 
### into seperate series'



drug_file.prod_ai = drug_file.prod_ai.astype(str)

indicies = drug_file[drug_file.prod_ai != 'nan'].index
indicies2 = drug_file[drug_file.prod_ai == 'nan'].index

present = drug_file.prod_ai.loc[indicies]
needs_work = drug_file.prod_ai.loc[indicies2]



In [ ]:
### Create DataFrame of 'nan' values, and attach teh series of matching indicies for which the drugname is present.
### Needs work...



needs_work_df = pd.DataFrame(needs_work)
needs_work_df['drugname'] = drug_file.drugname.loc[indicies2]

needs_work_df.drugname.value_counts()


# Custom Table Creation

In [ ]:
new_drug_df = drug_file[['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai', 'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr']]

drugs = pd.DataFrame()

In [ ]:
### for loop for determining the amount of 'PS' (Primary suspect), 'SS' (Secondary Suspects), 'C', (concamitants), 
### or 'I' (Interacting) medications present for every unique CaseID ***6.5hr runtime***

start_time = time.time()
p = 0
s = 0
c = 0
i = 0


for x,j in enumerate(new_drug_df.caseid.unique()):
    df = new_drug_df[new_drug_df.caseid == j]


    for y in range(len(df)):
        if df.iloc[y,3] == 'PS':
            p += 1
        elif df.iloc[y,3] == 'SS':
            s += 1
        elif df.iloc[y,3] == 'c':
            c += 1
        else:
            i += 1

    drugs.loc[x, 'CaseID'] = j
    drugs.loc[x, 'Primary'] = p
    drugs.loc[x, 'Secondary'] = s
    drugs.loc[x, 'Concamitant'] = c
    drugs.loc[x, 'Interacting'] = i
        
    p,s,c,i = 0,0,0,0
end_time = time.time()

total_time = (end_time - start_time) / 60
total_time / 60

# Mapping drugnames into numerical classes

In [ ]:
class_df = pd.DataFrame(columns=['drugname', 'class_id', 'class', 'indication'])
class_df.drugname = new_drug_df.drugname


In [ ]:
### Convert all values in 'prod_ai' column to string values, and then separate non-nan values out for class mapping



drug_file.prod_ai = drug_file.prod_ai.astype(str)
indicies = drug_file[drug_file.prod_ai != 'nan'].index
present = drug_file.prod_ai.loc[indicies]

In [ ]:
drug_file[['drugname', 'prod_ai']]

In [ ]:
present

In [ ]:

### using string methods 'startswith' and 'endswith', filter each product-active-ingredient to help map classes for each 
### medication   

for x,y in zip(present,indicies):
    if x.endswith('MAB') == True:
        class_df.loc[y, 'class_id'] = 1
        class_df.loc[y, 'class'] = 'monoclonal_antibody'
        class_df.loc[y, 'indication'] = ''
    elif x.endswith('PRIL'):
        class_df.loc[y, 'class_id'] = 2
        class_df.loc[y, 'class'] = 'ACE_inhibitor'
        class_df.loc[y, 'indication'] = 'hypertenstion' 
    elif x.endswith('TIDINE'):
        class_df.loc[y, 'class_id'] = 3
        class_df.loc[y, 'class'] = '2nd_gen_antihistamine'
        class_df.loc[y, 'indication'] = 'allergy'
    elif x.endswith('STATIN', 0, 12) or x.endswith('STATIN'):
        class_df.loc[y, 'class_id'] = 4
        class_df.loc[y, 'class'] = 'HMG-CoA reductase inhibitor'
        class_df.loc[y, 'indication'] = 'hyperlipidemia'
    elif x.endswith('AZEPAM') or x.endswith('ZOLAM'):
        class_df.loc[y, 'class_id'] = 5
        class_df.loc[y, 'class'] = 'benzodiazepine'
        class_df.loc[y, 'indication'] = 'anxiety'
    elif x.endswith('AFIL'):
        class_df.loc[y, 'class_id'] = 6
        class_df.loc[y, 'class'] = 'phosphodiesterase inhibitor'
        class_df.loc[y, 'indication'] = 'erectile dysfunction, hypertension'
    elif x.endswith('ANE'):
        class_df.loc[y, 'class_id'] = 7
        class_df.loc[y, 'class'] = 'inhaled anestetics'
        class_df.loc[y, 'indication'] = 'anesthesia'
    elif x.endswith('ARTAN'):
        class_df.loc[y, 'class_id'] = 8
        class_df.loc[y, 'class'] = 'angiotension receptor blocker'
        class_df.loc[y, 'indication'] = 'hypertension'
    elif x.endswith('AZINE'):
        class_df.loc[y, 'class_id'] = 9
        class_df.loc[y, 'class'] = 'phenothiazines'
        class_df.loc[y, 'indication'] = 'antipsychotic'
    elif x.endswith('AZOLE'):
        class_df.loc[y, 'class_id'] = 10
        class_df.loc[y, 'class'] = 'azole-antifungal'
        class_df.loc[y, 'indication'] = 'antifungal'

In [ ]:
class_df.class_id.value_counts()

In [ ]:
### change data type of CaseID from float to integer

drugs.CaseID = drugs.CaseID.astype(int)

### export dataframe to .csv file for manipulation later

drugs.to_csv('drugs.csv', index=False)

In [ ]:
### subset DataFrame with caseid and whether the reaction stopped when drug therapy was stopped (dechal) and if the reaction
### began again after drug therapy was restarted (rechal).


subside_relapse = drug_file[['caseid', 'dechal', 'rechal']]

In [ ]:
### The majority of the observations were of class 'Unknown', followed by 'Does Not Apply'
### There were 866404 null values also recorded


subside_relapse.dechal.value_counts()

In [ ]:
subside_relapse.dechal.isnull().sum()

In [ ]:
### The majority of the observations were of class 'Unknown', followed at distance by 'Negative' and 'Does Not Apply'
### There were 1600785 null values also recorded


subside_relapse.rechal.value_counts()

In [ ]:
subside_relapse.rechal.isnull().sum()

# Examining Primary and Secondary suspect frequency

In [ ]:
### Quantify each category

new_drug_df.role_cod.value_counts()

In [ ]:
### Separate Primary and Secondary suspects  for further investigation

primary = new_drug_df[new_drug_df.role_cod == 'PS']
secondary = new_drug_df[new_drug_df.role_cod == 'SS']

In [ ]:
primary

In [ ]:
### Choose features of interest to examine trends within each Primary and Secondary Suspect drug

primary = primary[['caseid', 'drugname', 'prod_ai', 'val_vbm', 'dose_vbm', 'cum_dose_chr']]
secondary = secondary[['caseid', 'drugname', 'dose_vbm']]

In [ ]:
### reset index

primary = primary.reset_index(drop=True)
secondary = secondary.reset_index(drop=True)

In [ ]:
### Interestingly enough, ZANTAC's generic formulation is referred to as RANITIDINE and they make up the top 2 spots on the 
### Primary suspect list. They may have high interactivity with other medications, or based on a patient's medication history,
### ZANTAC (RANITIDINE) is likely to interact with their other medications

primary.drugname.value_counts()

In [ ]:
### ZANTAC is also listed as the most frequent Secondary medication of interest. One could reasonably say that taking ZANTAC
### increases the likelihood of experiencing any SAE. 

secondary.drugname.value_counts()

In [ ]:
primary[['drugname', 'dose_vbm', 'cum_dose_chr']]